Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'pandas,': Expected end or semicolon (after name and no valid version specifier)
    pandas,
          ^


 # **01. Carga de Datos**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [1]:

import pandas as pd


#file_path = '/content/drive/MyDrive/AnalisisDatos/RIPS_20232024_HOSP.xlsx'
file_path = "/data/raw/RIPS_20232024_HOSP.xlsx"

(file_path)
try:
    df = pd.read_excel(file_path)
    print("Data loaded successfully!")
    display(df.head())
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

ModuleNotFoundError: No module named 'pandas'

# **02. Limpieza de Datos**


In [ ]:
# --- Validación de ID ---
# Normaliza por si vienen espacios o minúsculas
df['ID'] = df['ID'].astype('string').str.strip().str.upper()

# True si cumple ^PAC\d{5}$, False si no (y False también para NaN)
df['ID_valido'] = df['ID'].str.fullmatch(r'^PAC\d{5}$').fillna(False)


# --- Numérico para las llaves o códigos ---

for c in ['VIA INGRESO','CAUSA EXT','EDAD','UNIDAD EDAD','MUNICIPIO','GRUPO EDAD','AÑO']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce').astype('Int64')
df.head()

NameError: name 'df' is not defined

In [ ]:
import re

# --- Fechas ---
# Fechas (formato dd/mm/yyyy)
for c in ['Fecha_Ingreso','Fecha_Egreso']:
    df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=True)

# Validación de orden y no futuro
hoy = pd.Timestamp.today().normalize()
df.loc[df["Fecha_Ingreso"] > hoy, "Fecha_Ingreso"] = pd.NaT
df.loc[df["Fecha_Egreso"] > hoy, "Fecha_Egreso"] = pd.NaT
df.loc[(df["Fecha_Ingreso"].notna()) & (df["Fecha_Egreso"].notna()) &
       (df["Fecha_Egreso"] < df["Fecha_Ingreso"]), "Fecha_Egreso"] = pd.NaT

'''# --- Normalizaciones categóricas ---
map_sexo = {"HOMBRE":"M","H":"M","MASCULINO":"M","MUJER":"F","F":"F","FEMENINO":"F","X":"I","INTERSEX":"I"}
df["SEXO"] = df["SEXO"].replace(map_sexo)
df["SEXO"] = df["SEXO"].where(df["SEXO"].isin(["M","F","I"]), other=pd.NA)
map_unidad = {"AÑOS":"A","ANOS":"A","A":"A","MESES":"M","M":"M","DIAS":"D","D":"D"}
df["UNIDAD EDAD"] = df["UNIDAD EDAD"].replace(map_unidad)
df["UNIDAD EDAD"] = df["UNIDAD EDAD"].where(df["UNIDAD EDAD"].isin(["A","M","D"]), other=pd.NA)

# --- EDAD numérica ---
df["EDAD"] = pd.to_numeric(df["EDAD"], errors="coerce").astype("Int64")

# --- Normalizar EDAD a años (columna derivada) ---
def edad_a_anios(edad, unidad):
    if pd.isna(edad) or pd.isna(unidad):
        return np.nan
    if unidad == "A":
        return float(edad)
    if unidad == "M":
        return float(edad) / 12.0
    if unidad == "D":
        return float(edad) / 365.25
    return np.nan

df["Edad_Anios"] = [edad_a_anios(e,u) for e,u in zip(df["EDAD"], df["UNIDAD EDAD"])]

# --- Grupo de edad (ajusta cortes a tu estándar) ---
bins = [-0.001, 4, 11, 17, 44, 64, 200]
labels = ["0-4","5-11","12-17","18-44","45-64","65+"]
df["GRUPO EDAD"] = pd.cut(df["Edad_Anios"], bins=bins, labels=labels, right=True)
'''

# --- Validación CIE-10 ---
cie_regex = re.compile(r"^[A-TV-Z][0-9][0-9A-Z](\.[0-9A-Z]{1,2})?$")
cie_cols = ["Cod_Dx_Ppal_Egreso","DIAG EGRESO REL 1","DIAG EGRESO REL 2",
            "DIAG EGRESO REL 3","DIAG COMPLICACION","DIAG MUERTE"]

# Edad normalizada a años (derivada) — la FK será a dim_edad; aquí solo derivamos métrica
MAP_UNIDAD_EDAD = {'1':'A', '2':'M', '3':'D'}
df['UNIDAD_EDAD_TXT'] = df['UNIDAD EDAD'].astype('string').map(MAP_UNIDAD_EDAD)

def edad_a_anios(edad, unidad_txt):
    if pd.isna(edad) or pd.isna(unidad_txt):
        return np.nan
    if unidad_txt == 'A': return float(edad)
    if unidad_txt == 'M': return float(edad)/12.0
    if unidad_txt == 'D': return float(edad)/365.25
    return np.nan

df['Edad_Anios'] = [edad_a_anios(e, u) for e,u in zip(df['EDAD'], df['UNIDAD_EDAD_TXT'])]


# Duración de estancia (derivada)
df['Duracion_Dias'] = (df['Fecha_Egreso'] - df['Fecha_Ingreso']).dt.days
df.loc[df['Duracion_Dias'] < 0, 'Duracion_Dias'] = pd.NA


df.head()

,Cod_IPS,ID,VIA INGRESO,Fecha_Ingreso,CAUSA EXT,DIAGNOSTICO INGRESO,Cod_Dx_Ppal_Egreso,DIAG EGRESO REL 1,DIAG EGRESO REL 2,DIAG EGRESO REL 3,...,DIAG MUERTE,Fecha_Egreso,EDAD,UNIDAD EDAD,MUNICIPIO,GRUPO EDAD,SEXO,DEPARTAMENTO,AÑO,Duracion_Dias
0,52660212605,PAC00001,3,2023-09-04,13,R42X,F28X,NaN,NaN,NaN,...,NaN,2023-09-14,29,1,266,5,F,5,2023,10.0
1,52660212605,PAC00002,2,2023-02-05,13,R51X,A971,NaN,NaN,NaN,...,NaN,2023-02-28,22,1,266,4,M,5,2023,23.0
2,52660212605,PAC00003,3,2023-09-30,13,R688,B159,NaN,NaN,NaN,...,NaN,2023-09-30,22,1,1,4,F,5,2023,0.0
3,52660212605,PAC00004,3,2023-08-31,13,B24X,G042,NaN,NaN,NaN,...,NaN,2023-09-29,23,1,1,4,M,5,2023,29.0
4,52660212605,PAC00005,3,2023-08-26,2,T140,S822,NaN,NaN,NaN,...,NaN,2023-09-01,22,1,1,4,M,5,2023,6.0


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# División de los datos en conjuntos de entrenamiento y prueba
X = df.drop('Cod_Dx_Ppal_Egreso', axis=1)
y = df['Cod_Dx_Ppal_Egreso']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Error cuadrático medio: {mse}')

ValueError: could not convert string to float: 'PAC12893'